In [2]:
import pymorphy2
import codecs
import string
import re
import nltk
import ssl

from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>


False

In [4]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

lemmatizer = WordNetLemmatizer()
analyzer = pymorphy2.MorphAnalyzer()

MARKS = [',', '.', ':', '?', '«', '»', '-', '(', ')', '!', '\'', "—", ';', "”", "...", "\'\'", "/**//**/",
         "“", "„", "–"]



In [5]:
def parse_words_from_html(file):
    html_page = codecs.open(f"/Users/daniilkarpov/Desktop/Github/crawler/src/main/resources/crawler4j/{file}.txt", 'r', 'utf-8')
    
    soup = BeautifulSoup(html_page.read(), features='html.parser')
    for script in soup(
            ["script", "style", "meta", "link", "span", "a", "time", "button", "li", "dt", "h2", "h3", "legend"]):
        script.extract()  

    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text.split()

In [12]:
def tokenize():
    words = []
    with open("/Users/daniilkarpov/Desktop/Github/crawler/src/main/resources/crawler4j/index.txt", "r") as index:
        lines = index.readlines()
        numbers = [line[: line.find(" ")] for line in lines]
        for num in numbers:
            words.extend(parse_words_from_html(num))
        with open("lemmas.txt", "w", encoding="utf-8") as file:
            filtered_words = filter_words(words)
            for word in filtered_words:
                if len(word) > 2:
                    file.write(word + '\n')

In [7]:
def filter_words(words):
    words = list(filter(lambda word: (word not in string.punctuation) and (word not in MARKS), words))
    filtered_words = []
    for word in words:
        filtered_words.append("".join(filter(lambda char: char not in MARKS, word)))
    stop_words_rus = set(stopwords.words('russian'))
    stop_words_eng = set(stopwords.words('english'))
    filtered_words = [w.lower() for w in filtered_words if w not in stop_words_rus]
    filtered_words = [w.lower() for w in filtered_words if w not in stop_words_eng]
    filtered_words = filter(lambda word: word.isalpha(), filtered_words)
    return filtered_words

In [13]:
def lemmatize():
    with open("lemmas.txt", "r", encoding="utf-8") as words_list:
        words = words_list.readlines()
    lem_dict = {}
        
    rus_filter = re.compile("[а-яА-Я]+")
    eng_filter = re.compile("[a-zA-Z]+")
    rus_words = [w.strip() for w in filter(rus_filter.match, words)]
    eng_words = [w.strip() for w in filter(eng_filter.match, words)]
    for word in rus_words:
        normal_form = get_normal_form_rus(word)
        if normal_form:
            if normal_form not in lem_dict.keys():
                lem_dict[normal_form] = [word.strip()]
            else:
                if(word not in lem_dict[normal_form]):
                    lem_dict[normal_form].append(word.strip())
    for word in eng_words:
        normal_form = get_normal_form_eng(word)
        if normal_form:
            if normal_form not in lem_dict.keys():
                lem_dict[normal_form] = [word.strip()]
            else:
                if(word not in lem_dict[normal_form]):
                    lem_dict[normal_form].append(word.strip())

    file = open("lemmasAndTokens.txt", "w", encoding="utf-8")
    for word, tokens in lem_dict.items():
        file.write(f"{word}:")
        [file.write(f" {tok}") for tok in set(tokens)]
        file.write("\n")
    file.close()

In [9]:
def get_normal_form_rus(word):
    return analyzer.parse(word)[0].normal_form

In [10]:
def get_normal_form_eng(word):
    return lemmatizer.lemmatize(word)

In [14]:
tokenize()
lemmatize()